In [ ]:
!pip install -U "sentence-transformers[train]"

# Sentence Transformer MPNET
L'original és en anglès, però tot i així funciona prou bé el finetuning

In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [2]:
!export WANDB_MODE=disabled

In [5]:
from datasets import load_dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from torch.utils.data import DataLoader
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction

# 1. Load a model to finetune with (Optional) model card data
model = SentenceTransformer(
    "microsoft/mpnet-base",
    model_card_data=SentenceTransformerModelCardData(
        language="en",
        license="apache-2.0",
        model_name="MPNet base trained on semantic text similarity",
    )
)

# 2. Load the semantic text similarity dataset
dataset = load_dataset("projecte-aina/sts-ca")

# 3. Define a loss function (not needed for semantic similarity)
# You can skip this step as semantic similarity is not trained with a specific loss function.

# 4. Specify training arguments
args = SentenceTransformerTrainingArguments(
    output_dir="models/mpnet-base-sts-ca",
    num_train_epochs=40,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_ratio=0.1,
    fp16=True,
    bf16=False,
    report_to=None
)


# 5. (Optional) Create an evaluator & evaluate the base model
evaluator = EmbeddingSimilarityEvaluator(
    sentences1=dataset["validation"]["sentence1"],
    sentences2=dataset["validation"]["sentence2"],
    scores=dataset["validation"]["label"],
    main_similarity=SimilarityFunction.COSINE
)

# 6. Create a trainer & train
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    evaluator=evaluator,
)
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/532M [00:00<?, ?B/s]

Some weights of MPNetModel were not initialized from the model checkpoint at microsoft/mpnet-base and are newly initialized: ['mpnet.pooler.dense.bias', 'mpnet.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

TrainOutput(global_step=5200, training_loss=3.3695278754601112, metrics={'train_runtime': 976.2974, 'train_samples_per_second': 84.933, 'train_steps_per_second': 5.326, 'total_flos': 0.0, 'train_loss': 3.3695278754601112, 'epoch': 40.0})

In [6]:
train_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=dataset["train"]["sentence1"],
    sentences2=dataset["train"]["sentence2"],
    scores=dataset["train"]["label"],
    main_similarity=SimilarityFunction.COSINE
)

In [7]:
train_score = train_evaluator(model)
print(f"STS-CA train set Pearson correlation: {train_score['pearson_cosine']:.4f}")



STS-CA train set Pearson correlation: 0.9370


In [8]:
print(f"STS-CA train set Spearman correlation: {train_score['spearman_cosine']:.4f}")


STS-CA train set Spearman correlation: 0.9918


In [9]:
val_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=dataset["validation"]["sentence1"],
    sentences2=dataset["validation"]["sentence2"],
    scores=dataset["validation"]["label"],
    main_similarity=SimilarityFunction.COSINE
)


In [10]:
val_score = val_evaluator(model)
print(f"STS-CA validation set Pearson correlation: {val_score['pearson_cosine']:.4f}")
print(f"STS-CA validation set Spearman correlation: {val_score['spearman_cosine']:.4f}")


STS-CA validation set Pearson correlation: 0.5856
STS-CA validation set Spearman correlation: 0.5855


In [11]:
# 7. Evaluate the trained model on the test set
test_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=dataset["test"]["sentence1"],
    sentences2=dataset["test"]["sentence2"],
    scores=dataset["test"]["label"],
    main_similarity=SimilarityFunction.COSINE
)
test_score = test_evaluator(model)
print(f"STS-CA test set Pearson correlation: {test_score['pearson_cosine']:.4f}")
print(f"STS-CA test set Spearman correlation: {test_score['spearman_cosine']:.4f}")


STS-CA test set Pearson correlation: 0.6501
STS-CA test set Spearman correlation: 0.6820


In [12]:
model.save_pretrained("models/mpnet-base-sts-ca/final")

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

In [14]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) N
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [15]:
model.push_to_hub("finetuned-sts-ca-mpnet-base")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

'https://huggingface.co/pauhidalgoo/finetuned-sts-ca-mpnet-base/commit/7a3dff5ce1ecd1a42d2d5e9a2532af0d8fb9aeec'

# RoBERTA ca finetuning

In [16]:
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, models


word_embedding_model = models.Transformer('projecte-aina/roberta-base-ca-v2')

## Step 2: use a pool function over the token embeddings
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at projecte-aina/roberta-base-ca-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/848k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/506k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.21M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

Transformem el model a un Sentence Transformer (només hem fet un pooling)

In [17]:
model_roberta = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [18]:
# Load the semantic text similarity dataset
dataset = load_dataset("projecte-aina/sts-ca")

# 3. Define a loss function (not needed for semantic similarity)
# You can skip this step as semantic similarity is not trained with a specific loss function.

# 4. Specify training arguments
args = SentenceTransformerTrainingArguments(
    output_dir="models/roberta-ca",
    num_train_epochs=25,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_ratio=0.1,
    learning_rate=2e-5,
    weight_decay=0.01,
    fp16=True,
    bf16=False,
    report_to=None
)


# 5. (Optional) Create an evaluator & evaluate the base model
evaluator = EmbeddingSimilarityEvaluator(
    sentences1=dataset["validation"]["sentence1"],
    sentences2=dataset["validation"]["sentence2"],
    scores=dataset["validation"]["label"],
    main_similarity=SimilarityFunction.COSINE
)

# 6. Create a trainer & train
trainer = SentenceTransformerTrainer(
    model=model_roberta,
    args=args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    evaluator=evaluator,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for projecte-aina/sts-ca contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/projecte-aina/sts-ca
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
500,4.379800
1000,3.648600
1500,3.247900
2000,2.953900
2500,2.675300
3000,2.495500


Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

TrainOutput(global_step=3250, training_loss=3.172751220703125, metrics={'train_runtime': 664.8838, 'train_samples_per_second': 77.946, 'train_steps_per_second': 4.888, 'total_flos': 0.0, 'train_loss': 3.172751220703125, 'epoch': 25.0})

In [19]:
train_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=dataset["train"]["sentence1"],
    sentences2=dataset["train"]["sentence2"],
    scores=dataset["train"]["label"],
    main_similarity=SimilarityFunction.COSINE
)
train_score = train_evaluator(model_roberta)
print(f"STS-CA train set Pearson correlation: {train_score['pearson_cosine']:.4f}")
print(f"STS-CA train set Spearman correlation: {train_score['spearman_cosine']:.4f}")


STS-CA train set Pearson correlation: 0.9350
STS-CA train set Spearman correlation: 0.9899


In [20]:
val_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=dataset["validation"]["sentence1"],
    sentences2=dataset["validation"]["sentence2"],
    scores=dataset["validation"]["label"],
    main_similarity=SimilarityFunction.COSINE
)
val_score = val_evaluator(model_roberta)
print(f"STS-CA validation set Pearson correlation: {val_score['pearson_cosine']:.4f}")
print(f"STS-CA validation set Spearman correlation: {val_score['spearman_cosine']:.4f}")

STS-CA validation set Pearson correlation: 0.7185
STS-CA validation set Spearman correlation: 0.7312


In [21]:
# 7. Evaluate the trained model on the test set
test_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=dataset["test"]["sentence1"],
    sentences2=dataset["test"]["sentence2"],
    scores=dataset["test"]["label"],
    main_similarity=SimilarityFunction.COSINE
)
test_score = test_evaluator(model_roberta)
print(f"STS-CA test set Pearson correlation: {test_score['pearson_cosine']:.4f}")
print(f"STS-CA test set Spearman correlation: {test_score['spearman_cosine']:.4f}")


STS-CA test set Pearson correlation: 0.7429
STS-CA test set Spearman correlation: 0.7714


In [25]:
model_roberta.save_pretrained("models/roberta-ca/final")

Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

In [26]:
model_roberta.push_to_hub("finetuned-sts-roberta-base-ca-v2")

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

'https://huggingface.co/pauhidalgoo/finetuned-sts-roberta-base-ca-v2/commit/d81294c06c50549c8a81977c5cfc7262131e6cfb'